#setup environment

In [10]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

  Using cached PyYAML-5.1.tar.gz (274 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
fatal: destination path 'detectron2' already exists and is not an empty directory.


In [11]:
import torch, detectron2
import numpy as np
import cv2, os, json, random
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
from detectron2.config import get_cfg
from detectron2.model_zoo import get_config_file
from detectron2.engine import DefaultPredictor
from google.colab.patches import cv2_imshow
from detectron2.data import Metadata
torch.cuda.is_available()
setup_logger()

<Logger detectron2 (DEBUG)>

#load the google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#set the predictor cfg

In [ ]:
# Configuration
cfg = get_cfg()
cfg.merge_from_file(get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))  # Default configuration
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Number of classes
cfg.MODEL.DEVICE='cuda'
cfg.OUTPUT_DIR = "/content/drive/MyDrive/DIP_final_project/MaskRCNN/output_model"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # Path to the model weights
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75  # Set a threshold for this model
predictor = DefaultPredictor(cfg)

#load the metadata

In [ ]:
# 指定要读取的 JSON 文件路径
input_file_path = os.path.join(cfg.OUTPUT_DIR,"metadata.json")

# 尝试打开并读取文件
try:
    with open(input_file_path, 'r') as file:
        # 使用json.load从文件中加载数据
        metadata_dict = json.load(file)
        print("Metadata loaded successfully.")

        metadata = Metadata()
        metadata.__dict__.update(metadata_dict)

except FileNotFoundError:
    print(f"File {input_file_path} not found.")
except json.JSONDecodeError:
    print(f"Error decoding JSON in file {input_file_path}.")

#print the picture and save the mask

In [ ]:
import os

# 若資料夾不存在，則建立用來儲存 masks 的資料夾
mask_folder = '/content/drive/MyDrive/DIP_final_project/MaskRCNN/testcase_mask'
os.makedirs(mask_folder, exist_ok=True)

# 假設有20張圖片，依次處理 number+1
for i in range(1, 13):
    # 構造文件名，例如1.jpg、2.jpg、...、20.jpg
    image_filename = f"input{i}.jpg"
    image_path = os.path.join('/content/drive/MyDrive/DIP_final_project/MaskRCNN/testcase', image_filename)

    image = cv2.imread(image_path)
    outputs = predictor(image)

    # Visualize the prediction results
    v = Visualizer(image[:, :, ::-1], metadata, scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    print()

    # Display or save the result
    cv2_imshow(out.get_image()[:, :, ::-1])


    instances = outputs["instances"].to("cpu")
    masks = instances.get("pred_masks").numpy()

    # 將多個 masks 合併
    merged_mask = np.zeros_like(masks[0], dtype=np.uint8)
    for mask in masks:
        merged_mask = np.logical_or(merged_mask, mask)

    # 儲存合併後的 mask
    mask_filename = f"output{i}.jpg"
    mask_path = os.path.join(mask_folder, mask_filename)
    cv2.imwrite(mask_path, (merged_mask * 255).astype('uint8'))

    # 顯示合併後的 mask
    merged_mask_image = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    cv2_imshow(merged_mask_image)

print("Masks saved successfully.")